In [ ]:
from matplotlib import pyplot as plt
from matplotlib import gridspec
import matplotlib.axes as ax 
import numpy as np
import tensorflow as tf
from scipy import io
from tensorflow.keras.datasets import mnist, fashion_mnist, cifar10
from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder
import time
import os
import tensorflow.keras.backend as K
import pandas as pd
import Modules.utils as utils
import importlib
from scipy.stats import pearsonr
from scipy.cluster.hierarchy import cut_tree
from pprint import pprint
from collections import defaultdict

In [ ]:
importlib.reload(utils)
data_dir='../shared_folder'
writing_dir='../shared_folder'
# writing_dir='../'

Read file and datasets checks

In [ ]:
with np.load(f'{data_dir}/Bombyx/raw_data/X_long.npz') as f:
    long = f['reads']
with np.load(f'{data_dir}/Bombyx/raw_data/X_reads.npz') as f:
    reads = f['reads']

In [ ]:
total = 0
for i in range(len(long)):
    assert(len(long[i]) == len(long[i].rstrip()))
    total += len(long[i])
print(total)
total2 = 0
for i in range(len(reads)):
    assert(len(reads[i]) == len(reads[i].rstrip()))
    total2 += len(reads[i])
print(total2)

In [ ]:
with np.load(f'{data_dir}/HEK293-ZFAT/raw_data/Control_reads_deduped.npz') as f:
    reads = f['reads']
print(reads.shape)
sequences = utils.remove_reads_with_N(reads, tolerance=0)
print(len(sequences))

In [ ]:
data = 'CENPT'
model_name = 'model_inception2'
threshold = 0.75
with np.load(f'{data_dir}/{data}/results/{model_name}/seqs_{data}_over_{threshold}_with_{model_name}.npz') as f:
    reads = f['reads']
print(reads.shape)
sequences = utils.remove_reads_with_N(reads, tolerance=0)
print(len(sequences))

In [ ]:
dico = utils.check_read_lengths(reads)
dico = dict(sorted(dico.items(), reverse=True))
print('{read_length: nb_reads}')
print(dico)

In [ ]:
with np.load(f'{data_dir}/H3K9me3/dataset30M.npz') as f:
    reads = f['x_test']

In [ ]:
with np.load(f'{data_dir}/HEK293-ZFAT/raw_data/Control_reads_deduped.npz') as f:
    reads = f['reads']
print(reads.shape)
sequences = utils.remove_reads_with_N(reads, tolerance=0)
print(len(sequences))

print('Looking for read duplicates')
dico = utils.find_duplicates(reads, one_hot=False, batch_size=50_000_000, print_freq=20_000_000)
values = list(dico.values())
print('Looking for duplicate level duplicates')
dico2 = utils.find_duplicates(values, batch_size=100_000_000, print_freq=20_000_000)
dico2 = dict(sorted(dico2.items(), reverse=True))
print('{duplicate_level: nb_unique_reads}')
print(dico2)
print('top 5 duplicate level reads:', list(dico2.keys())[:5])

print('Looking for read duplicates')
dico = utils.find_duplicates(sequences, one_hot=False, batch_size=50_000_000, print_freq=20_000_000)
values = list(dico.values())
print('Looking for duplicate level duplicates')
dico2 = utils.find_duplicates(values, batch_size=100_000_000, print_freq=20_000_000)
dico2 = dict(sorted(dico2.items(), reverse=True))
print('{duplicate_level: nb_unique_reads}')
print(dico2)
print('top 5 duplicate level reads:', list(dico2.keys())[:5])

In [ ]:
selected_reads = [k for k, v in dico.items() if v == 72]
print(selected_reads)
print(dico[selected_reads[0]])

In [ ]:
# human telomere sequences
seq_list = [
    'CTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAAC',
    'CCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAA',
    'CCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTA',
    'ACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCT',
    'TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACC',
    'AACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCC',
    'GGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTA',
    'GTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAG',
    'TTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGG',
    'TAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGG',
    'AGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGT',
    'GGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTT'
]
for seq in seq_list:
    if seq in dico.keys():
        print(dico[seq], seq)
    else:
        print(0, seq)

In [ ]:
x = list(dico.keys())
y = list(dico.values())
plt.plot(x, y)
plt.yscale('log')
plt.xscale('log')
plt.ylabel('occurences')
plt.xlabel('duplicate level')
plt.show()
plt.close()

In [ ]:
with np.load(f'{data_dir}/H3K27me3/dataset_rel8.npz') as dataset:
    y_train = dataset['y_train']
    y_valid = dataset['y_valid']
    y_test = dataset['y_test']
print('total number of reads:',len(y_train) + len(y_valid) + len(y_test))
print('train:', len(y_train), 'valid:', len(y_valid), 'test:', len(y_test))
print('pos_train:', len(y_train[y_train == 1]), 'pos_valid', len(y_valid[y_valid == 1]), 'pos_test', len(y_test[y_test == 1]))

Plot test distribution

In [ ]:
data = 'CTCF'
dataset_name = 'dataset4'
model_name = 'model_inception2.2'
load_hist = False
hist_file = f'distrib_{model_name}_test_hist100000'
relabeled = False
new_labels = 'dataset_rel8'
single_pred_array = True
if load_hist:
    with np.load(f'{writing_dir}/{data}/results/{model_name}/{hist_file}.npz') as f:
        histIP = f['IP']
        histControl = f['Control']
    bins = np.linspace(0, 1, len(histIP)+1)
    win_start = 80000
    win_stop = 80100
    histIP, histControl, bins = histIP[win_start:win_stop], histControl[win_start:win_stop], bins[win_start:win_stop+1]
else:
    if single_pred_array:
        if relabeled:
            with np.load(f'{data_dir}/{data}/results/{model_name}/distrib_{model_name}_test.npz') as f:
                preds = f['pred']
            with np.load(f'{data_dir}//{data}/{new_labels}.npz') as f:
                y_test = f['y_test']
        else:
            with np.load(f'{data_dir}/{data}/results/{model_name}/distrib_{model_name}_test.npz') as f:
                preds = f['pred']
            with np.load(f'{data_dir}/{data}/{dataset_name}.npz') as f:
                y_test = f['y_test']
        predIP = preds[y_test == 1]
        predControl = preds[y_test == 0]
    else:
        with np.load(f'{data_dir}/{data}/results/distrib_{model_name}_test.npz') as f:
                predIP = f['predIP']
                predControl = f['predControl']
    accuracy = (len(predIP[predIP > 0.5]) + len(predControl[predControl < 0.5])) / (len(predIP) + len(predControl))
    n_bins = 100
    histIP, bins = np.histogram(predIP, bins=n_bins, range=(0, 1))
    histControl = np.histogram(predControl, bins=n_bins, range=(0, 1))[0]

widths = [5, 1]
heights = [1.2] + [2.5]*2
fig = plt.figure(figsize=(np.sum(widths), np.sum(heights)))
# fig.suptitle(f"Distribution of predictions of {model_name} on {data} for IP and Control reads")
gs = gridspec.GridSpec(ncols=len(widths), nrows=len(heights), width_ratios=widths, height_ratios=heights)

ax0_top = plt.subplot(gs[0, 0])
label_col0 = 'predictions'
ax0_top.annotate(label_col0, (0.1, 0.5), xycoords='axes fraction', va='center', fontsize=18)
ax0_top.axis('off')

ax1_top = plt.subplot(gs[0, 1])
label_col1 = 'scale'
ax1_top.annotate(label_col1, (0.1, 0.5), xycoords='axes fraction', va='center', fontsize=18)
ax1_top.axis('off')

for nrow in range(1,3):

    ax0 = plt.subplot(gs[nrow, 0])

    ax0.bar(bins[:-1], histIP, width=np.diff(bins), label='IP', alpha=0.5)
    ax0.bar(bins[:-1], histControl, width=np.diff(bins), label='Control', alpha=0.5)

    if not load_hist:
        ax0.axvline(x=0.5, color='black', label='t0.5')
        thres = 0.9
        ax0.axvline(x=thres, color='red', label=f't{thres}')
    
    ax1 = plt.subplot(gs[nrow, 1])
    if nrow == 1:
        label = 'linear'
        if not load_hist:
            ax1.annotate(f'accuracy: {round(accuracy, 4)}', (0, 0.4), xycoords='axes fraction', va='center', fontsize=13)
    elif nrow == 2:
        label = 'log'
        ax0.set_yscale('log')
    ax1.annotate(label, (0.1, 0.6), xycoords='axes fraction', va='center', fontsize=18)
    ax1.axis('off')
ax0.set_xlabel("predictions", fontsize=18)
ax0.set_ylabel("read count", fontsize=18)
handles, labels = ax0.get_legend_handles_labels()
ax0_top.legend(handles, labels, loc='right')

if not os.path.isdir(f'{writing_dir}/{data}/results/{model_name}'):
    os.makedirs(f'{writing_dir}/{data}/results/{model_name}')
fig_start = f'{writing_dir}/{data}/results/{model_name}/distrib_{model_name}_test'
fig_name = fig_start + '.png'
fig_dups = 0
while os.path.exists(fig_name):
    fig_dups += 1
    fig_name = fig_start + f'({fig_dups}).png'
plt.savefig(fig_name, bbox_inches='tight')
if not load_hist:
    np.savez(f'{writing_dir}/{data}/results/{model_name}/distrib_{model_name}_test_hist{n_bins}', IP=histIP, Control=histControl)
plt.show()
plt.close()

Select sequences

In [ ]:
with np.load(f'{data_dir}/{data}/{dataset_name}.npz') as f:
    x_test = f['x_test']
thres = 0.9
selected_reads = x_test[preds.ravel() > thres, :, :]
print(f'nb of selected reads with thres {thres}:', len(selected_reads))

In [ ]:
with np.load(f'{data_dir}/{data}/{dataset_name}.npz') as f:
    x_test = f['x_test']
thres0 = 0.83
thres1 = 0.99
preds = preds.ravel()
mask = np.logical_and(preds > thres0, preds < thres1)
selected_reads = x_test[mask, :, :]
print(f'nb of selected reads between {thres0} and {thres1}:', len(selected_reads))

Write into fasta file

In [ ]:
# 3m per 10_000_000 seqs of 101 bp
utils.write_fasta(utils.one_hot_to_seq(selected_reads), f'{writing_dir}/{data}/results/{model_name}/seqs_{data}_over_{thres}_with_{model_name}.fasta')

In [ ]:
# 3m per 10_000_000 seqs of 101 bp
utils.write_fasta(utils.one_hot_to_seq(selected_reads), f'{writing_dir}/{data}/results/{model_name}/seqs_{data}_over_{thres0}_below_{thres1}_with_{model_name}.fasta')

Plot training logs

In [ ]:
if not os.path.isdir(f'{writing_dir}/{data}/results/{model_name}'):
    os.mkdir(f'{writing_dir}/{data}/results/{model_name}/')
logs = pd.read_csv(f'{data_dir}/{data}/Trainedmodels/{model_name}/epoch_data.csv')
fig = plt.figure(figsize=(5, 6))
fig.suptitle(f'Training logs for {model_name} on {data}')
ax0 = plt.subplot(211)
ax0.plot(logs['val_accuracy'], label='valid')
ax0.plot(logs['accuracy'], label='train')
ax0.set_ylabel('accuracy')
ax0.legend()

ax1 = plt.subplot(212)
ax1.plot(logs['val_loss'], label='valid')
ax1.plot(logs['loss'], label='train')
ax1.set_xlabel('epoch')
ax1.set_ylabel('loss')
ax1.legend()
plt.legend()
plt.savefig(f'{writing_dir}/{data}/results/{model_name}/{model_name}_train_log.png', bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
IP_accuracy = np.size(predIP[predIP > 0.5]) / np.size(predIP)
Control_accuracy = (np.size(predControl[predControl < 0.5])
                    / np.size(predControl))
print('accuracy: ', (IP_accuracy + Control_accuracy) / 2)
print('IP accuracy: ', IP_accuracy)
print('Control accuracy: ', Control_accuracy)

Plot test distribution during epochs

In [ ]:
data = 'H3K27ac'
dataset_name = 'dataset4'
model_name = 'model_inception2.2_rel8'
new_labels = 'dataset_rel8'
relabeled = True
preds = np.load(f'{data_dir}/{data}/Trainedmodels/{model_name}/eval_epochs.npy')
# Watching for nan predictions
# for epoch in range(preds.shape[0]):
#     nans = np.count_nonzero(np.isnan(preds[epoch]))
#     print(f'Epoch {epoch}: {nans} nan predictions')
if relabeled:
    with np.load(f'{data_dir}/{data}/{new_labels}.npz') as f:
        y_test = f['y_test']
else:
    with np.load(f'{data_dir}//{data}/{dataset_name}.npz') as f:
        y_test = f['y_test']


predsIP = preds[:, y_test == 1]
predsControl = preds[:, y_test == 0]
correct_IP = (predsIP > 0.5)
correct_Control = (predsControl < 0.5)
accuracys = (np.sum(correct_IP, axis=1) + np.sum(correct_Control, axis=1)) / len(y_test)
amax = np.argmax(accuracys)
print(amax)
print(accuracys[amax])
plt.plot(accuracys)
plt.show()
plt.close()

# epochs = range(0, preds.shape[0], 5)
epochs = [0, 1, 2, 3, 4, 9, 14, 19]
widths = [5, 1]
heights = [1.2] + [2]*len(epochs)
fig = plt.figure(figsize=(np.sum(widths), np.sum(heights)))
# fig.suptitle(f"Distribution of predictions of {model_name} during epochs on {data} for IP and Control reads")
gs = gridspec.GridSpec(ncols=len(widths), nrows=len(heights), width_ratios=widths, height_ratios=heights)

ax0_top = plt.subplot(gs[0, 0])
label_col0 = 'prediction'
ax0_top.annotate(label_col0, (0.1, 0.5), xycoords='axes fraction', va='center', fontsize=18)
ax0_top.axis('off')

ax1_top = plt.subplot(gs[0, 1])
label_col1 = 'epoch'
ax1_top.annotate(label_col1, (0.1, 0.5), xycoords='axes fraction', va='center', fontsize=18)
ax1_top.axis('off')

nrow = 1
for epoch in epochs:
    ax0 = plt.subplot(gs[nrow, 0])
    ax0.hist(predsIP[epoch], bins=100, label='IP', alpha=0.5, range=(0,1))
    ax0.hist(predsControl[epoch], bins=100, label='Control', alpha=0.5, range=(0,1))
    ax0.axvline(x=0.5, color='black', label='base thres')
    ax0.set_yscale('log')

    ax1 = plt.subplot(gs[nrow, 1])
    label = f'{epoch+1}'
    ax1.annotate(label, (0.1, 0.5), xycoords='axes fraction', va='center', fontsize=18)
    ax1.annotate(f'accuracy: {round(accuracys[epoch], 4)}', (0, 0.4), xycoords='axes fraction', va='center', fontsize=13)
    ax1.axis('off')

    nrow += 1

handles, labels = ax0.get_legend_handles_labels()
ax0_top.legend(handles, labels, loc='right')
ax0.set_xlabel("predictions")
ax0.set_ylabel("read count")
if not os.path.isdir(f'{writing_dir}/{data}/results/{model_name}'):
    os.makedirs(f'{writing_dir}/{data}/results/{model_name}/')
plt.savefig(f'{writing_dir}/{data}/results/{model_name}/distrib_{model_name}_test_epochs.png', bbox_inches='tight')
plt.show()
plt.close()

Compare predictions between 2 models

In [ ]:
data = 'H3K27ac'
dataset_name = 'dataset4'
model_name = 'model_inception2.2_rel8'
model_name2 = 'model_inception2.2'
new_labels = 'dataset_rel8'
relabeled = True
preds = np.load(f'{data_dir}/{data}/Trainedmodels/{model_name}/eval_epochs.npy')
preds2 = np.load(f'{data_dir}/{data}/Trainedmodels/{model_name2}/eval_epochs.npy')
min_len = min(len(preds), len(preds2))
diff = preds[:min_len] - preds2[:min_len]

if relabeled:
    with np.load(f'{data_dir}/{data}/{new_labels}.npz') as f:
        y_test = f['y_test']
else:
    with np.load(f'{data_dir}//{data}/{dataset_name}.npz') as f:
        y_test = f['y_test']

diffIP = diff[:, y_test == 1]
diffControl = diff[:, y_test == 0]

# epochs = range(0, preds.shape[0], 5)
epochs = [0, 1, 2, 3, 4, 9, 14, 19]
widths = [5, 1]
heights = [1.2] + [2]*len(epochs)
fig = plt.figure(figsize=(np.sum(widths), np.sum(heights)))
# fig.suptitle(f"Distribution of prediction differences of {model_name} during epochs on {data} for IP and Control reads")
gs = gridspec.GridSpec(ncols=len(widths), nrows=len(heights), width_ratios=widths, height_ratios=heights)

ax0_top = plt.subplot(gs[0, 0])
label_col0 = 'prediction difference'
ax0_top.annotate(label_col0, (0.1, 0.5), xycoords='axes fraction', va='center', fontsize=18)
ax0_top.axis('off')

ax1_top = plt.subplot(gs[0, 1])
label_col1 = 'epoch'
ax1_top.annotate(label_col1, (0.1, 0.5), xycoords='axes fraction', va='center', fontsize=18)
ax1_top.axis('off')

nrow = 1
for epoch in epochs:
    ax0 = plt.subplot(gs[nrow, 0])
    ax0.hist(diffIP[epoch], bins=100, label='IP', alpha=0.5, range=(-1, 1))
    ax0.hist(diffControl[epoch], bins=100, label='Control', alpha=0.5, range=(-1, 1))
    # ax0.set_yscale('log')

    ax1 = plt.subplot(gs[nrow, 1])
    label = f'{epoch+1}'
    ax1.annotate(label, (0.1, 0.5), xycoords='axes fraction', va='center', fontsize=18)
    ax1.axis('off')

    nrow += 1

handles, labels = ax0.get_legend_handles_labels()
ax0_top.legend(handles, labels, loc='right')
ax0.set_xlabel("difference")
ax0.set_ylabel("read count")
if not os.path.isdir(f'{writing_dir}/{data}/results/{model_name}'):
    os.mkdir(f'{writing_dir}/{data}/results/{model_name}/')
plt.savefig(f'{writing_dir}/{data}/results/{model_name}/distrib_{model_name}_vs_{model_name2}_test_epochs.png', bbox_inches='tight')
plt.show()
plt.close()

Plot GC_content distribution

In [ ]:
data='HEK293-ZFAT'
dataset_name='dataset4'
with np.load(f'{data_dir}/{data}/{dataset_name}.npz') as f:
    x_test = f['x_test']
    y_test = f['y_test']
gc_content = utils.GC_content(x_test)

bins = np.histogram(gc_content, bins=100, range=(0, 1))[1]
accuracy, thres = utils.classify_1D(gc_content, y_test, bins=100)
print('accuracy :', accuracy)

widths = [5, 1]
heights = [1.2] + [2.5]*2
fig = plt.figure(figsize=(np.sum(widths), np.sum(heights)))
# fig.suptitle(f"Distribution of predictions of {model_name} on {data} for IP and Control reads")
gs = gridspec.GridSpec(ncols=len(widths), nrows=len(heights), width_ratios=widths, height_ratios=heights)

ax0_top = plt.subplot(gs[0, 0])
label_col0 = 'GC content'
ax0_top.annotate(label_col0, (0.1, 0.5), xycoords='axes fraction', va='center', fontsize=18)
ax0_top.axis('off')

ax1_top = plt.subplot(gs[0, 1])
label_col1 = 'scale'
ax1_top.annotate(label_col1, (0.1, 0.5), xycoords='axes fraction', va='center', fontsize=18)
ax1_top.axis('off')

for nrow in range(1,3):

    ax0 = plt.subplot(gs[nrow, 0])
    ax0.hist(gc_content[y_test == 1], bins, label='IP', alpha=0.5)
    ax0.hist(gc_content[y_test == 0], bins, label='Control', alpha=0.5)
    ax0.axvline(x=thres, color='black', label=f't{round(thres, 2)}')
    
    ax1 = plt.subplot(gs[nrow, 1])
    if nrow == 1:
        scale_label = 'linear'
        ax1.annotate(f'accuracy: {round(accuracy, 4)}', (0, 0.4), xycoords='axes fraction', va='center', fontsize=13)
    elif nrow == 2:
        scale_label = 'log'
        ax0.set_yscale('log')
    ax1.annotate(scale_label, (0.1, 0.6), xycoords='axes fraction', va='center', fontsize=18)
    ax1.axis('off')
ax0.set_xlabel("GC content", fontsize=18)
ax0.set_ylabel("read count", fontsize=18)
handles, labels = ax0.get_legend_handles_labels()
ax0_top.legend(handles, labels, loc='right')

fig.suptitle(f'Distribution of GC content in {data} reads', fontsize=18)
plt.savefig(f'{writing_dir}/{data}/results/gc_content_classification_{dataset_name}.png', bbox_inches='tight')
plt.show()
plt.close()

Load chromosome

In [ ]:
importlib.reload(utils)
chr_id = 1
# load one_hot_encoded sequence
chr_file = f'../shared_folder/Human/assembly/chr{chr_id}.npz'
with np.load(chr_file) as f:
    one_hot_chr = f['one_hot_genome']
window_size = 101
valid_window_idx = utils.remove_windows_with_N(one_hot_chr, window_size)
print('All windows:', len(one_hot_chr)-window_size+1)
print('Without N:', len(valid_window_idx))

# # load sequence with soft masked repeats
# with np.load(f'{data_dir}/Human/assembly/chr{chr_id}_seq.npz') as f:
#     chr_seq = f['reads'][0]
# chr_seq = np.array(list(chr_seq))
# repeats_idx, = np.where(chr_seq > 'Z')
# print(len(repeats_idx), 'bases in repeats')

# load repeat annotation
repeats = utils.parse_repeats(f'{data_dir}/Human/annotations/repeats/hg38.fa.out', window_size)
chr_repeats = repeats[f'{chr_id}']

Load predictions

In [ ]:
data = 'CTCF'
model_name = 'model_inception2.2'
model_name1 = 'model_inception2.2.rep1'
model_name_rel = 'model_inception2.2_rel8'
pred_on_genome = False
relabeled = False
if pred_on_genome:
    with np.load(f'{data_dir}/{data}/results/{model_name}/preds_on_genome.npz') as f:
        preds0 = f[f'chr{chr_id}']
    with np.load(f'{data_dir}/{data}/results/{model_name1}/preds_on_genome.npz') as f:
        preds1 = f[f'chr{chr_id}']
    if relabeled:
        with np.load(f'{data_dir}/{data}/results/{model_name_rel}/preds_on_genome.npz') as f:
            preds_rel = f[f'chr{chr_id}']
else:
    with np.load(f'{data_dir}/{data}/results/{model_name}/preds_on_chr{chr_id}.npz') as f:
        preds0 = f['preds']
    with np.load(f'{data_dir}/{data}/results/{model_name1}/preds_on_chr{chr_id}.npz') as f:
        preds1 = f['preds']
    if relabeled:
        with np.load(f'{data_dir}/{data}/results/{model_name_rel}/preds_on_chr{chr_id}.npz') as f:
            preds_rel = f['preds']

In [ ]:
importlib.reload(utils)
# z-score, mean and smooth
preds = (preds0 + preds1)/2  # mean
smooth_size = window_size
# smooth = utils.smooth(preds, smooth_size, mode='linear')
z_score_preds = utils.z_score(preds, valid_window_idx)
# z_score_preds0 = utils.z_score(preds0, valid_window_idx)
# z_score_preds1 = utils.z_score(preds1, valid_window_idx)
# z_score_preds_rel = utils.z_score(preds_rel, valid_window_idx)
# Smooth mean
smooth_size = window_size
z_score_smooth = utils.smooth(z_score_preds, smooth_size, mode='linear')
# z_score_smooth_rel = utils.smooth(z_score_preds_rel, smooth_size, mode='linear')

# # mean and std with or without windows containing Ns
# mean = np.mean(preds)
# std = np.std(preds)
# rel_preds = preds[valid_window_idx]
# rel_mean = np.mean(rel_preds)
# rel_std = np.std(rel_preds)
# print('mean and std for all windows:', mean, std)
# print('mean and std without Ns:', rel_mean, rel_std)

# # correlations between replicas
# inter_corr = pearsonr(preds0, preds1)
# print(inter_corr, 'correlation rep0-rep1')
# corr_mrel = pearsonr(preds, preds_rel)
# print(corr_mrel, 'correlation mean-relabeled')

Load peaks, logFC and p-value

In [ ]:
importlib.reload(utils)
files = {
    'CTCF': {
        'peaks': 'ENCFF748YHT.bed',
        'p_value': 'ENCFF701JJR.bigWig',
        'logFC': 'ENCFF902EIC.bigWig',
    },
    'H3K27ac': {
        'peaks': 'ENCFF553YTG.bed',
        'p_value': 'ENCFF472QZS.bigWig',
        'logFC': 'ENCFF408XBB.bigWig',
    },
    'H3K27me3': {
        'peaks': 'ENCFF968KLE.bed',
        'p_value': 'ENCFF805LJE.bigWig',
        'logFC': 'ENCFF463LIU.bigWig',
    },
    'H3K9me3': {
        'peaks': 'ENCFF411TKR.bed',
        'p_value': 'ENCFF042EJN.bigWig',
        'logFC': 'ENCFF088IKU.bigWig',
    },
}
peaks_dict = utils.parse_bed_peaks(f'{data_dir}/{data}/raw_data/peaks/{files[data]["peaks"]}', window_size)
p_vals = utils.load_annotation(f'{data_dir}/{data}/raw_data/p_value/{files[data]["p_value"]}', chr_id, window_size)
log_fc = utils.load_annotation(f'{data_dir}/{data}/raw_data/logFC/{files[data]["logFC"]}', chr_id, window_size)

# other_data = 'H3K27ac'
# other_peaks_dict = utils.parse_bed_peaks(f'{data_dir}/{other_data}/raw_data/peaks/{files[other_data]["peaks"]}', window_size)
# other_p_vals = utils.load_annotation(f'{data_dir}/{other_data}/raw_data/p_value/{files[other_data]["p_value"]}', chr_id, window_size)
# other_log_fc = utils.load_annotation(f'{data_dir}/{other_data}/raw_data/logFC/{files[other_data]["logFC"]}', chr_id, window_size)

In [ ]:
# Total peaks and only high ones
# adjust start and end to sliding window index
chr_peaks = peaks_dict[f'{chr_id}']
print(chr_peaks.shape, f'total peaks in {data}')
high_peaks = chr_peaks[chr_peaks[:,2] >= 1000]
print(high_peaks.shape, f'high peaks in {data}')
# # Peaks of different dataset
# other_chr_peaks = other_peaks_dict['1']
# print(other_chr_peaks.shape, f'total peaks in {other_data}')
# other_high_peaks = other_chr_peaks[other_chr_peaks[:,2] >= 1000]
# print(other_high_peaks.shape, f'high peaks in {other_data}')

# # Correlation between fold change and predicitons
# corr_mean = pearsonr(z_score_preds, log_fc)
# print(corr_mean, 'correlation mean-logFC')
# corr_smooth = pearsonr(z_score_smooth, log_fc)
# print(corr_smooth, 'correlation smooth-logFC')
# corr_rel = pearsonr(z_score_smooth_rel, log_fc)
# print(corr_rel, 'correlation rel-logFC')

# # Correlation with other dataset
# corr_FC = pearsonr(log_fc, other_log_fc)
# print(corr_FC, f'correlation logFC {data}-{other_data}')
# corr_p_val = pearsonr(p_vals, other_p_vals)
# print(corr_p_val, f'correlation p_vals {data}-{other_data}')

In [ ]:
# Correlation between p-value and predicitons
corr_mean = pearsonr(z_score_preds, p_vals)
print(corr_mean, 'correlation mean-p_value')
# corr_smooth = pearsonr(z_score_smooth, p_vals)
# print(corr_smooth, 'correlation smooth-p_value')
# corr_rel = pearsonr(z_score_smooth_rel, p_vals)
# print(corr_rel, 'correlation rel-p_value')

In [ ]:
# # Distribution of predictions on all windows
# plt.hist(preds, bins=100, range=(0, 1))
# plt.show()
# plt.close()
# # Distribution of predictions on N free windows
# plt.hist(preds[valid_window_idx], bins=100, range=(0, 1))
# plt.show()
# plt.close()
# # Same for z-score
# plt.hist(z_score_preds, bins=100)
# plt.show()
# plt.close()
plt.hist(z_score_smooth[valid_window_idx], bins=100, label='IP signal')
# plt.yscale('log')
plt.axvline(x=5, color='red', label='t5')
plt.legend()
plt.show()
plt.close()

In [ ]:
# Distribution of peak lengths
peak_lengths = np.diff(chr_peaks[:, :2], axis=1).ravel()
high_peaks_lengths = np.diff(high_peaks[:, :2], axis=1).ravel()
plt.hist(peak_lengths, bins=100, range=(0, 1500))
plt.show()
plt.close()
# plt.hist(high_peaks_lengths, bins=100)
# plt.show()
# plt.close()
# # Distribution of peak scores
# plt.hist(chr_peaks[:, 2], bins=100)
# plt.show()
# plt.close()

In [ ]:
# CTCF_strange_peaks = other_chr_peaks[peak_lengths == 810]
# print(CTCF_strange_peaks.shape, 'strange peaks')
# strange_peaks_indices = CTCF_strange_peaks[:, :1] + np.expand_dims(np.arange(810), axis=0)
# strange_peaks_one_hot = one_hot_chr[strange_peaks_indices]
# strange_peaks_seq = utils.one_hot_to_seq(strange_peaks_one_hot)
# utils.write_fasta(strange_peaks_seq, f'{data_dir}/CTCF/raw_data/peaks/strange_peaks_seqs.fasta')

In [ ]:
# Metaplot of predictions over mid peaks
importlib.reload(utils)
mid_peaks = (chr_peaks[:, 0] + chr_peaks[:, 1]) // 2
window_half_size = 600
clust, means, window = utils.metaplot_over_indices(z_score_preds, mid_peaks, window_half_size, plot='clustermap', res_dir=f'{writing_dir}/{data}/results/{model_name}', data=data, chr=f'chr{chr_id}')

In [ ]:
# Metaplot of predictions over mid peaks
importlib.reload(utils)
mid_peaks = (chr_peaks[:, 0] + chr_peaks[:, 1]) // 2
window_half_size = 600
corrs, means, window = utils.metaplot_over_indices(z_score_preds, mid_peaks, window_half_size, plot='heatmap', res_dir=f'{writing_dir}/{data}/results/{model_name}', data=data, chr=f'chr{chr_id}')

In [ ]:
n_clusters = 5
cluster_idx = cut_tree(clust.dendrogram_row.linkage, n_clusters).ravel()
clusters = [np.where(cluster_idx == i)[0] for i in range(n_clusters)]

clust_order = np.array(clust.dendrogram_row.reordered_ind)
samples = [np.where(clust_order == cluster[0])[0] for cluster in clusters]
print('cluster indices ordered from top to bottom', np.argsort(samples, axis=0).ravel())

mean_order = np.argsort(corrs)[::-1]
cluster_idx_mean_order = cluster_idx[mean_order]
plt.scatter(np.arange(len(mean_order)), cluster_idx_mean_order, marker='.')
plt.show()
plt.close()

In [ ]:
# plot of predictions over chosen peak
i = 0
peak_start, peak_end, score = chr_peaks[i]
print((peak_start, peak_end, score))
window = np.arange(peak_start-2000, peak_end+2000)
# both replicas
plt.plot(window, z_score_preds0[window], label='prediction rep0', alpha=0.5)
plt.plot(window, z_score_preds1[window], label='prediction rep1', alpha=0.5)
plt.plot([peak_start, peak_end], [2, 2], marker = 'o', label='ENCODE peak')
plt.legend(loc='lower right')
plt.show()
plt.close()
# mean vs smooth mean
plt.plot(window, z_score_preds[window], label='mean prediction')
plt.plot(window, z_score_smooth[window], label='smooth mean prediction')
plt.plot([peak_start, peak_end], [2, 2], marker = 'o', label='ENCODE peak')
plt.legend(loc='lower right')
plt.show()
plt.close()

In [ ]:
importlib.reload(utils)
# Find peaks in prediction with custom function and plot lengths
pred_thres = 1.5
pred_peaks = utils.find_peaks(z_score_smooth, pred_thres=pred_thres, length_thres=5, tol=0)
lengths = np.sort(np.diff(pred_peaks, axis=1).ravel())
distances = np.sort(pred_peaks[1:, 0] - pred_peaks[:-1, 1])  # check distances between peaks
print(pred_peaks.shape[0], 'peaks found')
MAX = 300
plt.hist(lengths, bins=100, range=(-0.5, MAX+0.5))
# plt.yscale('log')
# plt.xscale('log')
plt.show()
plt.close()

In [ ]:
importlib.reload(utils)
# find overlaps between predicted peaks and ENCODE peaks
(chr_overlap, pred_overlap), (chr_non_overlap, pred_non_overlap) = utils.overlapping_peaks(chr_peaks, pred_peaks)
print(len(chr_overlap), 'chr peaks overlap with pred peaks')
print(len(pred_overlap), 'pred peaks overlap with chr peaks')
print(len(chr_non_overlap), 'chr peaks do not overlap')
print(len(pred_non_overlap), 'pred peaks do not overlap')

In [ ]:
# plot predicted and encode peaks on predicted signal
importlib.reload(utils)
i = 13
peak_start, peak_end, *score = chr_peaks[i]
print((peak_start, peak_end, *score))
window = np.arange(peak_start-2000, peak_end+2000)
pred_peaks_to_show = utils.find_peaks_in_window(pred_peaks, peak_start-2000, peak_end+2000)
chr_peaks_to_show = utils.find_peaks_in_window(chr_peaks, peak_start-2000, peak_end+2000)

# plot peaks and smooth mean
ax1 = plt.subplot()
ax1.plot(window, z_score_smooth[window], label='smooth prediction')
ax2 = ax1.twinx()
ax2.plot(window, p_vals[window], label='p_value', alpha=0.5, color='brown')
for id, (start, end, *_) in enumerate(pred_peaks_to_show):
    if id == 0:
        ax1.plot([start, end], [pred_thres, pred_thres], marker = '.', color='red', label='predicted peaks')
    else:
        ax1.plot([start, end], [pred_thres, pred_thres], marker = '.', color='red')
for id, (start, end, *_) in enumerate(chr_peaks_to_show):
    if id == 0:
        ax1.plot([start, end], [pred_thres-1, pred_thres-1], marker = '.', color='black', label='ENCODE peaks')
    else:
        ax1.plot([start, end], [pred_thres-1, pred_thres-1], marker = '.', color='black')
handles1, labels1 = ax1.get_legend_handles_labels()
handles2, labels2 = ax2.get_legend_handles_labels()
plt.legend(handles1 + handles2, labels1 + labels2)
plt.show()
plt.close()

In [ ]:
# plot prediction signal vs fold change and p-value
# i = 3
peak_start, peak_end, *score = pred_non_overlap[i]
print(peak_start, peak_end, *score)
window = np.arange(peak_start-2000, peak_end+2000)
ax1 = plt.subplot()
ax1.plot(window, z_score_smooth[window], label='prediction', alpha=0.5)
ax1.plot([peak_start, peak_end], [pred_thres, pred_thres], marker = 'o', label='ENCODE peak')
ax2 = ax1.twinx()
ax2.plot(window, p_vals[window], label='p_value', alpha=0.5, color='black')
ax3 = ax1.twinx()
ax3.plot(window, log_fc[window], label='logFC', alpha=0.5, color='red')
handles1, labels1 = ax1.get_legend_handles_labels()
handles2, labels2 = ax2.get_legend_handles_labels()
handles3, labels3 = ax3.get_legend_handles_labels()
plt.legend(handles1 + handles2 + handles3, labels1 + labels2 + labels3, loc='lower right')
plt.show()
plt.close()

In [ ]:
print(chr_repeats.keys())
all_chr_repeats = np.array([rep for array in chr_repeats.values() for rep in array])
lengths = np.diff(all_chr_repeats, axis=1).ravel()
print(np.sum(lengths), 'bases in repeats')
print(len(one_hot_chr), f'bases in chr{chr_id}')

In [ ]:
pred_peaks_in_repeats = {}
for family in chr_repeats.keys():
    (pred_peaks_in_repeats[family], _), _ = utils.overlapping_peaks(pred_peaks, chr_repeats[family])
chr_peaks_in_repeats = {}
for family in chr_repeats.keys():
    (chr_peaks_in_repeats[family], _), _ = utils.overlapping_peaks(chr_peaks, chr_repeats[family])

In [ ]:
n_families = 0
for family in pred_peaks_in_repeats.keys():
    if '?' in family:
        print(family, len(pred_peaks_in_repeats[family]))
        n_families += 1
print(n_families, 'families')